1. 라이브러리 설치

In [ ]:
pip install selenium
pip install webdriver-manager

In [ ]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
import time
import random

2. 파일 불러오기

In [1]:
kosdaq = pd.read_excel('../dart_API/kosdaq_report_final.xlsx')
kosdaq_df = pd.DataFrame({'corp_name':kosdaq['회사명'], 'corp_num':kosdaq['사업자등록번호']})
print(kosdaq_df)

      corp_name    corp_num
0         블리츠웨이  6408701749
1         석경에이티  1348163122
2        지놈앤컴퍼니  6278700257
3       프리시젼바이오  3148608943
4           알체라  6438700337
...         ...         ...
1383     하이록코리아  6038106103
1384  SBI인베스트먼트  1208164573
1385   리더스 기술투자  1168116053
1386      엠벤처투자  1168115961
1387        모헨즈  3128115972

[1388 rows x 2 columns]


3. 함수 생성

In [ ]:
# 검색결과 리스트에 담는 함수
def result_to_list():
    srch_title = [i.text for i in driver.find_elements(By.CLASS_NAME, 'search_section_title')]     # 특허 명칭
    srch_text = [i.text for i in driver.find_elements(By.CLASS_NAME, 'search_txt')]     # 특허 내용
    srch_date = [i.text for i in driver.find_elements(By.XPATH, '//*[@id="mainsearch_info_list"]/div[1]/li[2]/span[2]/a')]     # 출원번호/일자
    srch_compname = [i.text for i in driver.find_elements(By.XPATH, '//*[@id="mainsearch_info_list"]/div[1]/li[3]/a/font')]     # 출원인
    
    # 검색결과 담기
    for k in range(0, len(srch_title)):
        info_dict = {}
        info_dict['corp_num'] = corp_num
        info_dict['corp_name'] = corp_name
        info_dict['count'] = driver.find_element(By.XPATH, '//*[@id="divMainArticle"]/form/section/div[1]/p/span[1]').text   # 총갯수
        info_dict['company_name'] = srch_compname[k]
        info_dict['title'] = srch_title[k]
        info_dict['text'] = srch_text[k]
        info_dict['date'] = srch_date[k]            
        patent_list.append(info_dict)

4. 특허정보검색서비스 KITRIS 크롤링

In [2]:
patent_list = []
error_list = []

# len(kosdaq_df['corp_num']) = 1388
for i in range(0, len(kosdaq_df['corp_num'])):

    corp_num = kosdaq_df['corp_num'][i]
    corp_name = kosdaq_df['corp_name'][i]

    try:
        # 검색페이지 진입
        url = "http://kpat.kipris.or.kr/kpat/searchLogina.do?next=MainSearch"
        driver = webdriver.Chrome(r'C:\Users\Chae\Downloads/chromedriver')
        driver.get(url)
        time.sleep(random.uniform(2,3))
        # 검색창 진입
        element = driver.find_element(By.ID, 'queryText')
        element.click()
        # 코스닥 사업자번호 입력
        element.send_keys(corp_num)
        element.send_keys(Keys.ENTER)
        time.sleep(random.uniform(2,3))

        # 총 검색결과 갯수
        total = driver.find_element(By.XPATH, '//*[@id="divMainArticle"]/form/section/div[1]/p/span[1]').text
        total = total.replace(',','')
        # total = int(total)
        print(total)

        # 검색결과가 있으면, '페이지당 90개' 클릭
        if len(driver.find_elements(By.CLASS_NAME, 'search_section_title')) > 0:
            icon_page = driver.find_element(By.XPATH, '//*[@id="opt28"]')
            icon_page.click()
            icon_show_90 = driver.find_element(By.XPATH, '//*[@id="opt28"]/option[3]')
            icon_show_90.click()
            icon_go = driver.find_element(By.XPATH, '//*[@id="pageSel"]/a/img')
            icon_go.click()
            time.sleep(random.uniform(1,2))

        # 검색결과 리스트 담기
        result_to_list()
        # print(patent_list)
        print('***** {}번째 기업 정상수집 p1 *****'.format(i+1))


        # 페이지 이동하면서 리스트 담기
        try:
            for i in range(1, 10):
                icon_next = driver.find_element(By.XPATH, '//*[@id="divBoardPager"]/a[{}]'.format(i))
                icon_next.click()
                time.sleep(random.uniform(2,3))
                result_to_list()
                print('***** {}페이지 완료 *****'.format(i+1))
            print('***** 모든 페이지 완료 *****', '\n')
        except:
            pass
        finally:
            # print(patent_list)
            driver.quit()

    except:
        print('!!!!! Error 발생 : {}, {} !!!!!'.format(corp_name, corp_num))
        error_dict = {}
        error_dict['corp_num'] = corp_num
        error_dict['corp_name'] = corp_name
        error_list.append(error_dict)
        driver.quit()

# DataFrame으로 변환
df_patent = pd.json_normalize(patent_list)
df_patent.to_csv('C:/Users/Chae/Desktop/SeSAC/Project/5. ESG_prj/1. 데이터 수집/patent_crawling/patent_list2.csv', index=False)
df_patent.to_excel('C:/Users/Chae/Desktop/SeSAC/Project/5. ESG_prj/1. 데이터 수집/patent_crawling/test2.xlsx', index=False)

df_error = pd.json_normalize(error_list)
df_error.to_csv('C:/Users/Chae/Desktop/SeSAC/Project/5. ESG_prj/1. 데이터 수집/patent_crawling/error_list2.csv', index=False)


C:\Users\Chae\AppData\Local\Temp\ipykernel_12480\4088631858.py:41: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(r'C:\Users\Chae\Downloads/chromedriver')


2
***** 1번째 기업 정상수집 p1 *****
75
***** 2번째 기업 정상수집 p1 *****
24
***** 3번째 기업 정상수집 p1 *****
119
***** 4번째 기업 정상수집 p1 *****
***** 1페이지 완료 *****
11
***** 5번째 기업 정상수집 p1 *****
242
***** 6번째 기업 정상수집 p1 *****
***** 1페이지 완료 *****
***** 2페이지 완료 *****
9
***** 7번째 기업 정상수집 p1 *****
!!!!! Error 발생 : 티엘비, 1348677072 !!!!!
11
***** 9번째 기업 정상수집 p1 *****
16
***** 10번째 기업 정상수집 p1 *****
!!!!! Error 발생 : DB금융스팩8호, 4978701922 !!!!!
29
***** 12번째 기업 정상수집 p1 *****
78
***** 13번째 기업 정상수집 p1 *****
4
***** 14번째 기업 정상수집 p1 *****
14
***** 15번째 기업 정상수집 p1 *****
29
***** 16번째 기업 정상수집 p1 *****
154
***** 17번째 기업 정상수집 p1 *****
***** 1페이지 완료 *****
32
***** 18번째 기업 정상수집 p1 *****
74
***** 19번째 기업 정상수집 p1 *****
26
***** 20번째 기업 정상수집 p1 *****
1
***** 21번째 기업 정상수집 p1 *****
62
***** 22번째 기업 정상수집 p1 *****
54
***** 23번째 기업 정상수집 p1 *****
35
***** 24번째 기업 정상수집 p1 *****
67
***** 25번째 기업 정상수집 p1 *****
97
***** 26번째 기업 정상수집 p1 *****
***** 1페이지 완료 *****
!!!!! Error 발생 : 피플바이오, 2148718925 !!!!!
24
***** 28번째 기업 정상수집 p1 *****
67
***** 2